<a href="https://colab.research.google.com/github/faridkharis/skripsi/blob/main/2_Topic_Modelling_(LDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install --upgrade numpy Interval

In [2]:
# pip install --upgrade gensim

In [3]:
# pip install pyldavis==3.2.1

In [4]:
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
path = 'drive/MyDrive/Colab Notebooks/App Improvement Recommendation System/Data/'

In [7]:
df = pd.read_csv(path + 'data-ulasan-oy-preprocessed.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,id,review,rating,tanggal
0,1a4f7075-a672-474a-8012-bfe0ae10f60d,"['pakai', 'saldo', 'uang', 'rekening', 'kerja'...",5,2023-02-03 07:50:45
1,0b266aa9-8a3c-45ee-accd-00f21288dcde,"['yydsllnmmmmm', 'ugggffffersal']",4,2023-02-03 06:36:36
2,2e9da359-2a38-4817-ab46-38f1e4961c4e,['bagus'],5,2023-02-02 19:32:06
3,eed6744d-6515-4242-98f4-b2871c35b923,"['sungguh', 'sayang', 'daftar', 'pilih', 'plat...",1,2023-02-02 11:50:32
4,8e110f99-22c6-4026-8545-84a6e6d8566c,"['mudah', 'nyaman', 'aman', 'fiturnya', 'lengk...",5,2023-01-29 09:17:44
...,...,...,...,...
15974,b3fea17c-6a49-433b-8386-5cfb87a44815,[],5,2017-09-29 04:27:02
15975,a8d2ed88-f990-40df-811f-93c5bb001a02,['lgsung'],5,2017-09-28 02:25:33
15976,36ac9248-5074-4710-9448-3efa21b40ea5,['keren'],5,2017-09-27 13:18:46
15977,4f4894db-9bbb-4982-b832-40f98f0927e0,['bagus'],5,2017-09-27 05:10:22


# Tokenizing #2

In [8]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
import nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [9]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['review'] = df['review'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['review'] = df['review'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['review'] = df['review'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['review'] = df['review'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['review'] = df['review'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['review'] = df['review'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['review'] = df['review'].apply(word_tokenize_wrapper)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
df

,id,review,rating,tanggal
0,1a4f7075-a672-474a-8012-bfe0ae10f60d,"[pakai, saldo, uang, rekening, kerja, langsung...",5,2023-02-03 07:50:45
1,0b266aa9-8a3c-45ee-accd-00f21288dcde,"[yydsllnmmmmm, ugggffffersal]",4,2023-02-03 06:36:36
2,2e9da359-2a38-4817-ab46-38f1e4961c4e,[bagus],5,2023-02-02 19:32:06
3,eed6744d-6515-4242-98f4-b2871c35b923,"[sungguh, sayang, daftar, pilih, platform, tra...",1,2023-02-02 11:50:32
4,8e110f99-22c6-4026-8545-84a6e6d8566c,"[mudah, nyaman, aman, fiturnya, lengkap, laku,...",5,2023-01-29 09:17:44
...,...,...,...,...
15974,b3fea17c-6a49-433b-8386-5cfb87a44815,[],5,2017-09-29 04:27:02
15975,a8d2ed88-f990-40df-811f-93c5bb001a02,[lgsung],5,2017-09-28 02:25:33
15976,36ac9248-5074-4710-9448-3efa21b40ea5,[keren],5,2017-09-27 13:18:46
15977,4f4894db-9bbb-4982-b832-40f98f0927e0,[bagus],5,2017-09-27 05:10:22


# LDA

In [15]:
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2dense

In [16]:
from gensim.models import CoherenceModel

documents = df['review']

def determine_optimal_num_topics(documents, min_num_topics=4, max_num_topics=10):
    best_num_topics = 0
    best_coherence_score = -1

    for num_topics in range(min_num_topics, max_num_topics + 1):
        id2word = Dictionary(documents)
        corpus = [id2word.doc2bow(doc) for doc in documents]
        lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)
        coherence_model = CoherenceModel(model=lda_model, texts=documents, dictionary=id2word, coherence='c_v')
        coherence_score_ = coherence_model.get_coherence()
        print("Topic ", num_topics, " : ", coherence_score_)
        
        if coherence_score_ > best_coherence_score:
            best_coherence_score = coherence_score_
            best_num_topics = num_topics
            
    return best_num_topics

# Example usage
num_topics = determine_optimal_num_topics(documents)
print("Optimal number of topics: ", num_topics)

Topic  4  :  0.39252845704531497


Topic  5  :  0.3222951470879581


Topic  6  :  0.31176405023748927


Topic  7  :  0.38573216548959344


Topic  8  :  0.3478352596660695


Topic  9  :  0.42667031930211163


Topic  10  :  0.38792724932043043
Optimal number of topics:  9


In [17]:
# import numpy as np

# documents = df['review']

# def determine_optimal_num_topics(documents, min_num_topics=4, max_num_topics=10):
#     best_num_topics = 0
#     best_perplexity = np.inf

#     for num_topics in range(min_num_topics, max_num_topics + 1):
#         id2word = Dictionary(documents)
#         corpus = [id2word.doc2bow(doc) for doc in documents]
#         lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)
#         perplexity = lda_model.log_perplexity(corpus)
#         print("Topic ", num_topics, " : ", perplexity)

#         if perplexity < best_perplexity:
#             best_perplexity = perplexity
#             best_num_topics = num_topics

#     return best_num_topics

# # Example usage
# num_topics = determine_optimal_num_topics(documents)
# print("Optimal number of topics: ", num_topics)

In [18]:
# Train an LDA model on the bag of words representation
documents = df['review']
id2word = Dictionary(documents)
corpus = [id2word.doc2bow(doc) for doc in documents]

lda = LdaModel(corpus, num_topics=num_topics, id2word=id2word)

lda.show_topics(num_topics=num_topics, num_words=8)

[(0,
  '0.045*"bangkrut" + 0.037*"transfer" + 0.030*"admin" + 0.027*"bank" + 0.025*"harga" + 0.022*"biaya" + 0.021*"update" + 0.014*"banget"'),
 (1,
  '0.112*"pulsa" + 0.109*"poin" + 0.034*"chat" + 0.032*"tukar" + 0.021*"tolong" + 0.017*"redem" + 0.016*"masuk" + 0.016*"reedem"'),
 (2,
  '0.043*"minimal" + 0.041*"masuk" + 0.035*"jelek" + 0.020*"kartu" + 0.019*"top" + 0.017*"turunin" + 0.016*"up" + 0.014*"tolong"'),
 (3,
  '0.187*"bagus" + 0.127*"point" + 0.059*"oke" + 0.022*"manfaat" + 0.022*"banget" + 0.021*"bintang" + 0.018*"naikin" + 0.011*"kasih"'),
 (4,
  '0.039*"verifikasi" + 0.035*"mudah" + 0.027*"susah" + 0.024*"buka" + 0.021*"download" + 0.014*"teman" + 0.014*"masuk" + 0.012*"minimal"'),
 (5,
  '0.163*"mantap" + 0.019*"pelit" + 0.019*"minimal" + 0.018*"telkomsel" + 0.013*"nambah" + 0.011*"masak" + 0.010*"pesan" + 0.008*"pakai"'),
 (6,
  '0.101*"kasih" + 0.071*"bantu" + 0.066*"terima" + 0.018*"terimakasih" + 0.018*"bintang" + 0.017*"tingkat" + 0.016*"moga" + 0.012*"kecewa"'),
 (


Topic Label


0.   Topic 0 : biaya admin (mahal) - User Needs
1.   Topic 1 : penukaran pulsa (susah) - Functional Specifications
2.   Topic 2 : top up (mahal) - User Needs
3.   Topic 3 : poin (bagus) - User Needs
4.   Topic 4 : verifikasi (susah) - Functional Specifications
5.   Topic 5 : operator seluler (mantap) - User Needs
6.   Topic 6 : aplikasi (membantu) - User Needs
7.   Topic 7 : transaksi (cepat) - Intection Design
8.   Topic 8 : penukaran pulsa (susah) - Functional Specifications



In [19]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda.show_topics(formatted=False)
data_flat = [w for w_list in documents for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count']) 
df_imp_wcount

,word,topic_id,importance,word_count
0,bangkrut,0,0.045480,201
1,transfer,0,0.037198,1537
2,admin,0,0.029720,493
3,bank,0,0.027472,1039
4,harga,0,0.025437,169
...,...,...,...,...
85,tolong,8,0.018739,763
86,admin,8,0.015746,493
87,masuk,8,0.015586,897
88,lumayan,8,0.013820,148


In [20]:
term_dict = {}

for document in df['review']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))

9876


In [21]:
for i in range(len(df)):
        a=df.iloc[i][2]
        document.append(a)
        
document[0:5]

['like', 'it', 5, 4, 5]

In [22]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['topik', 'persentase topik', 'kata kunci']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [23]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=corpus, texts=documents)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['no', 'topik', 'persentase topik', 'kata kunci', 'teks']

In [24]:
df_dominant_topic.head(10)

,no,topik,persentase topik,kata kunci,teks
0,0,7.0,0.3647,"keren, banget, suka, moga, nyesel, cepat, suks...","[pakai, saldo, uang, rekening, kerja, langsung..."
1,1,3.0,0.7037,"bagus, point, oke, manfaat, banget, bintang, n...","[yydsllnmmmmm, ugggffffersal]"
2,2,3.0,0.5556,"bagus, point, oke, manfaat, banget, bintang, n...",[bagus]
3,3,6.0,0.4769,"kasih, bantu, terima, terimakasih, bintang, ti...","[sungguh, sayang, daftar, pilih, platform, tra..."
4,4,7.0,0.6030,"keren, banget, suka, moga, nyesel, cepat, suks...","[mudah, nyaman, aman, fiturnya, lengkap, laku,..."
5,5,2.0,0.4237,"minimal, masuk, jelek, kartu, top, turunin, up...","[bank, bsi]"
6,6,6.0,0.5285,"kasih, bantu, terima, terimakasih, bintang, ti...","[kasih, bintang, erti]"
7,7,5.0,0.3935,"mantap, pelit, minimal, telkomsel, nambah, mas...","[virtual, accountnyamundur]"
8,8,5.0,0.3842,"mantap, pelit, minimal, telkomsel, nambah, mas...","[mantap, pelayanannys]"
9,9,7.0,0.7778,"keren, banget, suka, moga, nyesel, cepat, suks...","[keren, moga, masyarakat]"


In [25]:
df = df.join(df_dominant_topic['topik'])
df = df.join(df_dominant_topic['persentase topik'])
df = df.join(df_dominant_topic['kata kunci'])
df.head()

,id,review,rating,tanggal,topik,persentase topik,kata kunci
0,1a4f7075-a672-474a-8012-bfe0ae10f60d,"[pakai, saldo, uang, rekening, kerja, langsung...",5,2023-02-03 07:50:45,7.0,0.3647,"keren, banget, suka, moga, nyesel, cepat, suks..."
1,0b266aa9-8a3c-45ee-accd-00f21288dcde,"[yydsllnmmmmm, ugggffffersal]",4,2023-02-03 06:36:36,3.0,0.7037,"bagus, point, oke, manfaat, banget, bintang, n..."
2,2e9da359-2a38-4817-ab46-38f1e4961c4e,[bagus],5,2023-02-02 19:32:06,3.0,0.5556,"bagus, point, oke, manfaat, banget, bintang, n..."
3,eed6744d-6515-4242-98f4-b2871c35b923,"[sungguh, sayang, daftar, pilih, platform, tra...",1,2023-02-02 11:50:32,6.0,0.4769,"kasih, bantu, terima, terimakasih, bintang, ti..."
4,8e110f99-22c6-4026-8545-84a6e6d8566c,"[mudah, nyaman, aman, fiturnya, lengkap, laku,...",5,2023-01-29 09:17:44,7.0,0.6030,"keren, banget, suka, moga, nyesel, cepat, suks..."


In [26]:
# df['topik'] = df['topik'].astype('string')
# type(df['topik'][0])
# df['topik'] = df['topik'].str.replace('0.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('1.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('2.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('3.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('4.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('5.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('6.0', 'Fitur penukaran pulsa')
# df['topik'] = df['topik'].str.replace('7.0', 'Fitur penukaran pulsa')
# df.head()

In [27]:
# df['kata kunci'] = df['kata kunci'].str.replace('admin, harga, biaya, mahal, kasih, bank, transfer, terima, bohong, bayar', 'biaya admin mahal')
# df['kata kunci'] = df['kata kunci'].str.replace('1', '')
# df['kata kunci'] = df['kata kunci'].str.replace('2', '')
# df['kata kunci'] = df['kata kunci'].str.replace('suka, parah, mantab, banget, senang, baguss, si, muas, sia, bikin', '')
# df['kata kunci'] = df['kata kunci'].str.replace('4', '')
# df['kata kunci'] = df['kata kunci'].str.replace('5', '')
# df['kata kunci'] = df['kata kunci'].str.replace('6', '')
# df['kata kunci'] = df['kata kunci'].str.replace('7', '')
# df['kata kunci'] = df['kata kunci'].str.replace('8', '')
# df['kata kunci'] = df['kata kunci'].str.replace('9', '')
# df['kata kunci'] = df['kata kunci'].str.replace('moga, transaksi, mudah, cepat, sukses, bantu, capek, pending, respon, tarik', '')
# df['kata kunci'] = df['kata kunci'].str.replace('11', '')
# df['kata kunci'] = df['kata kunci'].str.replace('tambahin, tolong, susah, ribu, sangat, besar, the, mimin, ambil, terimakasih', '')
# df['kata kunci'] = df['kata kunci'].str.replace('13', '')
# df['kata kunci'] = df['kata kunci'].str.replace('kasih, bintang, terima, verifikasi, melulu, kali, masuk, gagal, ktp, biar', '')
# df['kata kunci'] = df['kata kunci'].str.replace('15', '')
# df['kata kunci'] = df['kata kunci'].str.replace('bagus, manfaat, kartu, mudah, moga, bodoh, buang, hadiah, kembang, lancar', '')
# df['kata kunci'] = df['kata kunci'].str.replace('lumayan, uninstal, nambah, kirim, masak, kemarin, muncul, tertawa, chat, tunggu', '')
# df

In [28]:
# save to drive
with open(path + 'data-ulasan-oy-topic.csv', 'w') as f:
  df.to_csv(f)